<a href="https://colab.research.google.com/github/beaumontslim/misc/blob/main/txt_classfification_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [ ]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/Restaurant_Reviews.tsv.txt', delimiter= '\t', quoting = 3)

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

corpus = []

import re

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  1000 non-null   object
 1   Liked   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [ ]:
for i in range(0, 1000):

  customer_review = re.sub('[^a-zA-Z]', ' ',dataset['Review'][i])
  customer_review = customer_review.lower()
  customer_review = customer_review.split()
  clean_review = [ps.stem(word) for word in customer_review if not word in set(stopwords.words('english'))]
  clean_review = ' '.join(clean_review)
  corpus.append(clean_review)

In [ ]:
#corpus[0]

corpus[9]

#corpus[12]

'great touch'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 1500, min_df = 3, max_df = 0.6)

X = vectorizer.fit_transform(corpus).toarray()

X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
y = dataset.iloc[:, 1].values

y[0]

1

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

import torch
import torch.nn as nn
from torch.nn import functional as F


In [ ]:
Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()

ytrain_ = torch.from_numpy(y_train)
ytest_ = torch.from_numpy(y_test)

Xtrain_.shape, ytrain_.shape

(torch.Size([800, 467]), torch.Size([800]))

In [ ]:
Xtest_.shape, ytest_.shape

(torch.Size([200, 467]), torch.Size([200]))

In [ ]:
#input suze reflects shape of Xtest shape, feeds to two output neurons (for 1/0)

input_size = 467
output_size = 2
hidden_size = 500

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = torch.nn.Linear(input_size, hidden_size)
    self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
    self.fc3 = torch.nn.Linear(hidden_size, output_size)

  def forward (self, X):
    X = torch.relu((self.fc1(X)))
    X = torch.relu((self.fc2(X)))
    X = self.fc3(X)

    return F.log_softmax(X,dim=1)

model = Net()

import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters(), lr = .01)
loss_fn = nn.NLLLoss()



epochs = 100

In [ ]:
for epoch in range (epochs):
  optimizer.zero_grad()
  Ypred = model(Xtrain_)
  loss = loss_fn(Ypred, ytrain_)
  loss.backward()
  optimizer.step()
  print('Epock! Text_Class_PyTorch', epoch, 'loss',loss.item())

Epock! Text_Class_PyTorch 0 loss 0.6934382915496826
Epock! Text_Class_PyTorch 1 loss 0.6715011596679688
Epock! Text_Class_PyTorch 2 loss 0.5349559187889099
Epock! Text_Class_PyTorch 3 loss 0.34322109818458557
Epock! Text_Class_PyTorch 4 loss 0.20937415957450867
Epock! Text_Class_PyTorch 5 loss 0.15478485822677612
Epock! Text_Class_PyTorch 6 loss 0.09776280075311661
Epock! Text_Class_PyTorch 7 loss 0.08123435825109482
Epock! Text_Class_PyTorch 8 loss 0.06858418136835098
Epock! Text_Class_PyTorch 9 loss 0.0519028939306736
Epock! Text_Class_PyTorch 10 loss 0.045633625239133835
Epock! Text_Class_PyTorch 11 loss 0.047879111021757126
Epock! Text_Class_PyTorch 12 loss 0.040305525064468384
Epock! Text_Class_PyTorch 13 loss 0.03422447293996811
Epock! Text_Class_PyTorch 14 loss 0.03582938015460968
Epock! Text_Class_PyTorch 15 loss 0.035591404885053635
Epock! Text_Class_PyTorch 16 loss 0.03425924479961395
Epock! Text_Class_PyTorch 17 loss 0.03057049587368965
Epock! Text_Class_PyTorch 18 loss 0.03

In [ ]:
sample = ['good batting by America']
sample = vectorizer.transform(sample).toarray()
#sample
torch.from_numpy(sample).float()
sentiment = model(torch.from_numpy(sample).float())
sentiment

tensor([[-1.8671, -0.1679]], grad_fn=<LogSoftmaxBackward>)

In [ ]:
sample = ['bad performance by the jayhawks, no sweet sixteen! :/']
sample = vectorizer.transform(sample).toarray()
#sample
torch.from_numpy(sample).float()
sentiment = model(torch.from_numpy(sample).float())
sentiment

tensor([[-0.8582, -0.5515]], grad_fn=<LogSoftmaxBackward>)

In [ ]:
model.state_dict()

OrderedDict([('fc1.weight',
              tensor([[-0.0162, -0.0042,  0.0076,  ..., -0.0130, -0.0045, -0.0105],
                      [-0.0409, -0.0167, -0.0258,  ..., -0.0336, -0.0094, -0.0455],
                      [ 0.0040, -0.0878, -0.0813,  ..., -0.0733,  0.0214, -0.0862],
                      ...,
                      [ 0.1107,  0.0252, -0.0643,  ...,  0.0858, -0.0873,  0.0238],
                      [-0.0420, -0.0490, -0.0175,  ...,  0.0149, -0.0366, -0.0583],
                      [-0.1135,  0.0088,  0.0499,  ...,  0.1269,  0.1399, -0.0911]])),
             ('fc1.bias',
              tensor([-0.0321, -0.0371, -0.0633, -0.0541, -0.0130, -0.0901, -0.0212,  0.0240,
                       0.0047,  0.0564, -0.0203, -0.0177,  0.0115,  0.0017,  0.0247, -0.0132,
                      -0.0097, -0.0778,  0.0587,  0.0245, -0.0473,  0.0085,  0.0120,  0.0082,
                      -0.0292,  0.0294, -0.0518, -0.0485, -0.0504,  0.0010, -0.0205,  0.0010,
                       0.0256,  0.00

In [ ]:
torch.save(model.state_dict(),'text_class_pytorch')

In [ ]:
!ls

sample_data  text_class_pytorch


In [ ]:
from google.colab import files

In [ ]:
files.download('text_class_pytorch')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pickle
with open ('tfidfmodel.pickle', 'wb') as file:
  pickle.dump(vectorizer,file)